In [1]:
import pandas as pd
import plotly.graph_objects as go

In [2]:
dat = pd.read_csv('../alexailyin_23:05:00_tweets.csv')

In [28]:
class Summary:

    def __init__(self, filename):
        self.dat = pd.read_csv(filename)
        self.wc_dict = None

    def summary_stats(self, type, stat):
        stats = self.dat.describe().transpose().iloc[1:, :]
        try:
            return stats.loc[type][stat]
        except KeyError as e:
            raise KeyError('Possible columns include {}'.format(list(stats.columns)))

    def get_most_likes(self, clean=True):
        row = self.dat[self.dat['likes']==self.dat['likes'].max()]
        tw_url = 'twitter.com/' + row['user_id'].values.flat[0] + '/status/' + str(row['tweet_id'].values.flat[0])
        return tw_url

    def get_most_re(self, clean=True):
        row = self.dat[self.dat['likes']==self.dat['likes'].max()]
        tw_url = 'twitter.com/' + row[['user_id']] + \
             '/status/' + str(row[['tweet_id']])
        return tw_url

    def likes_retweets_plot(self):
        fig = go.Figure(data=[
                    go.Bar(name='Tweets', x=['Likes', 'Retweets'], y=[self.dat['likes'].sum(), self.dat['retweets'].sum()])
              ])
        fig.update_layout(
            title="Total Likes vs. Retweets",
            yaxis_title='Sum',
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            width=400,
            height=400
        )
        return fig.to_html(include_plotlyjs='cdn')

    def wc(self, clean=True):
        col = 'clean_text'
        if not clean:
            col = 'text'

        wc = {}
        for tweet in self.dat[col]:
            for word in tweet.split():
                if word not in wc:
                    wc[word] = 1
                else:
                    wc[word] += 1

        wc = {k: v for k, v in sorted(wc.items(), key=lambda item: item[1], reverse=True)}
        self.wc_dict = wc

    def wc_plot(self, count=25):
        if not self.wc_dict:
            self.wc_dict()

        fig = go.Figure(data=[
            go.Bar(name='Word Count', x=list(self.wc_dict.keys())[:count], y=list(self.wc_dict.values())[:count])
        ])
        fig.update_xaxes(tickangle=45)
        fig.update_layout(
            title="Top {} Words".format(count),
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            width=450,
            height=450
        )
        return fig.to_html(include_plotlyjs='cdn')



In [29]:
s = Summary('../alexailyin_23:05:00_tweets.csv')

In [31]:
s.get_most_likes()

'twitter.com/ClaudiaCereal/status/1275562213969809408'

In [60]:
from selenium import webdriver

In [50]:
url = 'https://publish.twitter.com/?query=' + s.get_most_likes().replace('/', '%2F') + '&theme=dark&widget=Tweet'

In [53]:
url

'https://publish.twitter.com/?query=twitter.com%2FClaudiaCereal%2Fstatus%2F1275562213969809408&theme=dark&widget=Tweet'

In [68]:
browser = webdriver.Chrome()
browser.get(url)
x = browser.find_element_by_class_name('EmbedCode-code')
print(x.text)

<blockquote class="twitter-tweet" data-theme="dark"><p lang="en" dir="ltr">Grocery store and retail workers should be getting double pay during these times. Because these folks have been doing the MOST in their establishments. <a href="https://t.co/yR53BkSjHa">https://t.co/yR53BkSjHa</a></p>&mdash; 👸🏾L E A👸🏾 (@_MissLeandra) <a href="https://twitter.com/_MissLeandra/status/1274556477857030148?ref_src=twsrc%5Etfw">June 21, 2020</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>


In [ ]:
page.